My SQL Sales Analysis Code

In [22]:
import sqlite3
import pandas as pd 

database = sqlite3.connect("sales_Analysis.db.sqlite")

cursor = database.cursor()

GENERAL QUERIES

In [21]:
# View first 5 rows
df= pd.read_sql_query("SELECT * FROM sales_table LIMIT 5;", database)

df

,Product_ID,Sale_Date,Sales_Rep,Region,Sales_Amount,Quantity_Sold,Product_Category,Unit_Cost,Unit_Price,Customer_Type,Discount,Payment_Method,Sales_Channel,Region_and_Sales_Rep
0,1052,2023-02-03,Bob,North,5053.97,18,Furniture,152.75,267.22,Returning,0.09,Cash,Online,North-Bob
1,1093,2023-04-21,Bob,West,4384.02,17,Furniture,3816.39,4209.44,Returning,0.11,Cash,Retail,West-Bob
2,1015,2023-09-21,David,South,4631.23,30,Food,261.56,371.4,Returning,0.2,Bank Transfer,Retail,South-David
3,1072,2023-08-24,Bob,South,2167.94,39,Clothing,4330.03,4467.75,New,0.02,Credit Card,Retail,South-Bob
4,1061,2023-03-24,Charlie,East,3750.2,13,Electronics,637.37,692.71,New,0.08,Credit Card,Online,East-Charlie


In [23]:
# Check Column Names and Types
cursor.execute("PRAGMA table_info(sales_table);")
columns = cursor.fetchall()
for col in columns:
    print(col)

(0, 'Product_ID', 'TEXT', 0, None, 0)
(1, 'Sale_Date', 'TEXT', 0, None, 0)
(2, 'Sales_Rep', 'TEXT', 0, None, 0)
(3, 'Region', 'TEXT', 0, None, 0)
(4, 'Sales_Amount', 'TEXT', 0, None, 0)
(5, 'Quantity_Sold', 'TEXT', 0, None, 0)
(6, 'Product_Category', 'TEXT', 0, None, 0)
(7, 'Unit_Cost', 'TEXT', 0, None, 0)
(8, 'Unit_Price', 'TEXT', 0, None, 0)
(9, 'Customer_Type', 'TEXT', 0, None, 0)
(10, 'Discount', 'TEXT', 0, None, 0)
(11, 'Payment_Method', 'TEXT', 0, None, 0)
(12, 'Sales_Channel', 'TEXT', 0, None, 0)
(13, 'Region_and_Sales_Rep', 'TEXT', 0, None, 0)


In [24]:
# Show how many rows are in the dataset
cursor.execute("SELECT COUNT(*) FROM sales_table;")
print("Total rows:", cursor.fetchone()[0])


Total rows: 1000


PERFORMANCE ANALYSIS

In [25]:
# Total Sales By Region
cursor.execute(""" 
        SELECT Region, ROUND(SUM(Sales_Amount),0) AS Total_Sales
        FROM sales_table
        GROUP BY Region
        ORDER BY Total_Sales Desc; """)

region_sales = cursor.fetchall()
for index, row in enumerate(region_sales):
    if index > 0: 
        print("---")
    print(f"Region: {row[0]}\nAmount Sold: {row[1]}")

Region: North
Amount Sold: 1369613.0
---
Region: East
Amount Sold: 1259793.0
---
Region: West
Amount Sold: 1235609.0
---
Region: South
Amount Sold: 1154251.0


In [26]:
# Find Top 5 Sales Reps
cursor.execute("""
        SELECT Sales_Rep, ROUND(SUM(Sales_Amount),2) AS Total_Sales_By_Rep
        FROM sales_table
        GROUP BY Sales_Rep
        ORDER BY Total_Sales_By_Rep Desc
        LIMIT 5; """)                          # In this dataset, there are only 5 sales reps, so only 5 will be shown.

sales_reps_results = cursor.fetchall()
for index, row in enumerate(sales_reps_results):
    if index > 0: 
        print("---")
    print(f"Sales Rep: {row[0]}\nAmount Sold: {row[1]}")


Sales Rep: David
Amount Sold: 1141737.36
---
Sales Rep: Bob
Amount Sold: 1080990.63
---
Sales Rep: Eve
Amount Sold: 970183.99
---
Sales Rep: Alice
Amount Sold: 965541.77
---
Sales Rep: Charlie
Amount Sold: 860811.48


In [27]:
# Since David has the highest sales amount amongst the sales reps, we will look over his performance over the 2023 year.
# Rank David's Sales Over 2023
cursor.execute("""
    SELECT strftime('%m', Sale_Date) AS Month, 
           ROUND(SUM(Sales_Amount), 0)
    FROM sales_table
    WHERE Sales_Rep = 'David' AND strftime('%Y', Sale_Date) = '2023'         
    GROUP BY Month
    ORDER BY Month; """)
                                                        # Can Change Year to 2024 to find David's 2024 performance


cust_rep_yr = cursor.fetchall()
print("David's sales Performance By Month For 2023:\n")
print(f"{'Month':<10}{'Sales Amount'}")
print("-" * 30)

for index, row in enumerate(cust_rep_yr):
    print(f"{row[0]:<10}{row[1]}")


David's sales Performance By Month For 2023:

Month     Sales Amount
------------------------------
01        62713.0
02        89292.0
03        85578.0
04        80145.0
05        74809.0
06        69234.0
07        79501.0
08        99625.0
09        62550.0
10        192511.0
11        112730.0
12        124671.0


In [28]:
# Find Sales in the North region, by David, throughout the month of October of 2023. All three are the most profitable region, sales rep, and month for said sales rep.
cursor.execute("""
        SELECT Product_ID, Sales_Amount, Quantity_Sold
               FROM sales_table
               WHERE Sales_Rep = 'David'
                AND strftime('%Y', Sale_Date) = '2023' 
                AND strftime('%m', Sale_Date) = '10'
                AND Region = "North"
               ORDER BY Sales_Amount Desc;
""")

cust_rep_stats = cursor.fetchall()
print("David's sales Performance In October 2023:\n")
print(f"{'Product ID':<15}{'Sales Amount':<15}{'Quantity Sold'}")
print("-" * 45)

for index, row in enumerate(cust_rep_stats):
    print(f"{row[0]:<15}{row[1]:<15}{row[2]}")

David's sales Performance In October 2023:

Product ID     Sales Amount   Quantity Sold
---------------------------------------------
1035           9476.2         1
1034           8919.47        34
1095           8828.74        21
1089           7751.92        43
1052           7444.77        46
1089           7405.38        15
1036           6499.94        49
1053           6033.09        1
1002           5842.88        30
1012           4384.88        34
1025           3422.54        44
1033           1115.42        43


CUSTOMER ANALYSIS

In [30]:
# Find Percentage of Sales that are Returning and New Customers
cursor.execute("""
    SELECT Customer_Type, COUNT(*) AS Total_Sales, ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM sales_table),1) AS Percentage
    FROM sales_table
    GROUP BY Customer_Type;               
""")

customer_type_per = cursor.fetchall()
for index, row in enumerate(customer_type_per):
    if index > 0: 
        print("---")
    print(f"Customer Type: {row[0]}\nAmount of Sales: {row[1]}\nPercentage of Total Sales: {row[2]}")



Customer Type: New
Amount of Sales: 504
Percentage of Total Sales: 50.4
---
Customer Type: Returning
Amount of Sales: 496
Percentage of Total Sales: 49.6


In [32]:
# Find Average sales amount per customer type
cursor.execute("""
    SELECT Customer_Type, ROUND(AVG(Sales_Amount), 0) AS Avg_Sales
    FROM sales_table
    GROUP BY Customer_Type;
""")

avg_sales_result = cursor.fetchall()
for index, row in enumerate(avg_sales_result):
    if index > 0: 
        print("---")
    print(f"Customer Type: {row[0]}\nAverage Sales Amount: {row[1]}")

Customer Type: New
Average Sales Amount: 4973.0
---
Customer Type: Returning
Average Sales Amount: 5067.0


In [33]:
# Find Frequency of Purchases by Customer Type and Product Category
cursor.execute("""
    SELECT Customer_Type, Product_Category, COUNT(*) AS Purchase_Frequency
    FROM sales_table
    GROUP BY Customer_Type, Product_Category
    ORDER BY Purchase_Frequency DESC;                    
""")                            # Can swap ORDER BY to Customer_Type to split New/Returning customers

frequency_purchases = cursor.fetchall()
print("Frequency Purchases:\n")
print(f"{'Customer Type':<18}{'Product Category':<18}{'Frequency'}")
print("-" * 45)

for index, row in enumerate(frequency_purchases):
    print(f"{row[0]:<18}{row[1]:<18}{row[2]}")

Frequency Purchases:

Customer Type     Product Category  Frequency
---------------------------------------------
Returning         Furniture         146
New               Clothing          143
New               Electronics       136
Returning         Clothing          125
Returning         Food              115
New               Furniture         114
New               Food              111
Returning         Electronics       110


In [37]:
#Find Preffered Payment Type
cursor.execute("""
    SELECT Payment_Method, Customer_Type, ROUND(SUM(Sales_Amount),2) AS Payment_Sales
    FROM sales_table
    GROUP BY Payment_Method, Customer_Type
    ORDER BY Payment_Sales DESC;
""")

Payment_Type = cursor.fetchall()
for index, row in enumerate(Payment_Type):
    if index > 0: 
        print("---")
    print(f"Payment Method: {row[0]}\nCustomer Type: {row[1]}\nTransaction Amount: ${row[2]}")

Payment Method: Credit Card
Customer Type: Returning
Transaction Amount: $920894.22
---
Payment Method: Bank Transfer
Customer Type: New
Transaction Amount: $889778.5
---
Payment Method: Credit Card
Customer Type: New
Transaction Amount: $836669.3
---
Payment Method: Bank Transfer
Customer Type: Returning
Transaction Amount: $827584.67
---
Payment Method: Cash
Customer Type: New
Transaction Amount: $779810.5
---
Payment Method: Cash
Customer Type: Returning
Transaction Amount: $764528.04


PRODUCT ANALYSIS

In [38]:
# Find which products sell the most
cursor.execute("""
    SELECT Product_Category, SUM(Quantity_Sold) AS Total_Quantity_Sold
    FROM sales_table
    GROUP BY Product_Category
    ORDER BY Total_Quantity_Sold DESC; """)

product_info = cursor.fetchall()
for index, row in enumerate(product_info):
    if index > 0: 
        print("---")
    print(f"Product Category: {row[0]}\nProduct Sold: {row[1]}")

Product Category: Clothing
Product Sold: 6922
---
Product Category: Furniture
Product Sold: 6729
---
Product Category: Electronics
Product Sold: 6096
---
Product Category: Food
Product Sold: 5608


In [39]:
# Find Which Products Cost The Most
cursor.execute("""
    SELECT Product_Category, ROUND(SUM(Unit_Cost),0) AS Total_Cost
    FROM sales_table
    GROUP BY Product_Category
    ORDER BY Total_Cost DESC;
""")

product_cost = cursor.fetchall()
for index, row in enumerate(product_cost):
    if index > 0: 
        print("---")
    print(f"Product Category: {row[0]}\nTotal Product Cost: ${row[1]}")

Product Category: Clothing
Total Product Cost: $662117.0
---
Product Category: Furniture
Total Product Cost: $642828.0
---
Product Category: Electronics
Total Product Cost: $626151.0
---
Product Category: Food
Total Product Cost: $544208.0


In [40]:
# Find cost per unit for each product category
cursor.execute("""
    SELECT Product_Category,
       ROUND(AVG(Unit_Cost),2) AS Avg_Unit_Cost
    FROM sales_table
    GROUP BY Product_Category
    ORDER BY Avg_Unit_Cost DESC;         
""")

product_avg_cost = cursor.fetchall()
for index, row in enumerate(product_avg_cost):
    if index > 0: 
        print("---")
    print(f"Product Category: {row[0]}\nAvg Unit Cost: ${row[1]}")

Product Category: Electronics
Avg Unit Cost: $2545.33
---
Product Category: Furniture
Avg Unit Cost: $2472.42
---
Product Category: Clothing
Avg Unit Cost: $2470.59
---
Product Category: Food
Avg Unit Cost: $2408.0


In [42]:
# Find The Most Profitable Product
cursor.execute("""
    SELECT Product_Category, ROUND(SUM(((Unit_Price - Unit_Cost) * Quantity_Sold) - (Discount * Quantity_Sold)),0) AS Product_Profit        
    FROM sales_table
    GROUP BY Product_Category
    ORDER BY Product_Profit DESC;
""" )

product_profit = cursor.fetchall()
for index, row in enumerate(product_profit):
    if index > 0: 
        print("---")
    print(f"Product Category: {row[0]}\nProduct Profit: ${row[1]}")

Product Category: Furniture
Product Profit: $1778375.0
---
Product Category: Clothing
Product Profit: $1711878.0
---
Product Category: Electronics
Product Profit: $1573477.0
---
Product Category: Food
Product Profit: $1420264.0
